In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [3]:
links = ['https://inreallife.my/real-people/',
        'https://inreallife.my/real-relationships/',
        'https://inreallife.my/real-mental-health/',
        'https://inreallife.my/real-skills/']

In [11]:
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 


for i in tqdm(links):

    x = 1

    while True:

        while True:
            try:
                r = requests.get(f'{i}page/{x}',headers = headers)
                break
            except Exception as e:
                print(e)
                time.sleep(1.0)
                
        soup = BeautifulSoup(r.text, "lxml")
        
        div = soup.find_all('div', attrs = {"class":"post_header_title"})

        if len(div) == 0:
            break

        for link in soup.find_all('div', attrs = {"class":"post_header_title"}):
            
            a = link.find('h5')
            href = a.find('a').get('href')
            hrefs.append(href)
        
        x = x + 1
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [33:54<00:00, 508.51s/it]


In [39]:
len(hrefs)

1219

In [38]:
hrefs = list(set(hrefs))

In [15]:
with open('inreallife-link.json', 'a') as f:
    json.dump(hrefs, f)

In [36]:
with open('inreallife-link.json') as f:
    hrefs = json.load(f)

In [44]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        title = soup.find('div', class_="page_title_inner")
        headline = title.find("h1").text
        h = soup.find('div', attrs = {"class":"post_wrapper"})
        p = [p.text for i, p in enumerate(h.find_all("p")) if i >= 2 and p.text and not (any(a.get("href") for a in p.find_all("a")) or "https" in p.text)]

    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [45]:
max_workers = 5

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('inreallife.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1219/1219 [42:43<00:00,  2.10s/it]
